In [10]:
import pandas as pd
from kiteconnect import KiteConnect

In [28]:
#API Creds
api_key = "jlqcagntp2b2pz1d"  
api_secret = "ph8nb7ale6oiym2upkuxlx7incp7p5mc"
access_token = "VMVyUb4rqBP5p0hmNHdfgpwI2069Sex5"
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

In [35]:
# Load input CSV file
input_df = pd.read_csv("stocks.csv")

In [36]:
input_df.head(10)
type(input_df)

pandas.core.frame.DataFrame

In [37]:
# Output DataFrame
output_columns = ['Date', 'Symbol', 'BuySignal', 'ExitSignal']
output_df = pd.DataFrame(columns=output_columns)

In [38]:
# EMA function
def calculate_ema(data, window):
    weights = (2 / (window + 1)) * pd.Series(range(1, window + 1)).apply(lambda x: (1 - (2 / (window + 1))) ** x)
    ema = data.rolling(window=window, min_periods=1).apply(lambda x: (weights * x).sum())
    return ema

In [39]:
date =[]
token=[]
for index, row in input_df.iterrows():
    date.append(row['Date'])
    token.append(row['Token'])


['2020-08-03', '2020-08-03', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-17', '2020-08-24', '2020-08-24', '2020-08-24', '2020-08-24', '2020-08-24', '2020-08-24', '2020-08-24', '2020-08-31', '2020-09-14', '2020-09-14', '2020-09-21', '2020-10-05', '2020-10-05', '2020-10-12', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-19', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-10-26', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020-11-02', '2020

In [46]:
# Fetch historical data
# historical_data = kite.historical_data(token,date,date,"day")
historical_data =[]

for i in range(0,len(date)):
    try:
         historical_data.append(kite.historical_data(token[i],date[i],date[i],"day"))
    except ReadTimeout:
	    pass
   


    
# Fetch historical data for the next week using Kite API
next_week_date = pd.to_datetime(date) + pd.DateOffset(weeks=1)
next_week_date_str = next_week_date.strftime('%Y-%m-%d')
    
historical_data_next_week= []
for i in range(0, len(next_week_date_str)):
    try:
         historical_data_next_week.append(kite.historical_data(token[i],next_week_date_str[i],next_week_date_str[i],"day"))
    except ReadTimeout:
	    pass
    

# Extract closing prices
# closing_prices = [day['close'] for day in historical_data_next_week]
# closing_series = pd.Series(closing_prices)

NameError: name 'ReadTimeout' is not defined

In [ ]:
ema_window = 20
ema = calculate_ema(closing_series, ema_window)
last_ema = ema.iloc[-1]
    
# Check if the closing price of the next week's candle touches the EMA 20 level
if closing_prices[-1] >= last_ema:
        buy_signal = 'Yes'
        # Check if the closing price of the next week's candle remains above 95% of the original closing price
        exit_price = 0.95 * original_closing_price
        if min(closing_prices) >= exit_price:
            exit_signal = 'Yes'
else:
            exit_signal = 'No'
else:
        buy_signal = 'No'
        exit_signal = 'No'


In [ ]:
#Output DataFrame
    output_df = output_df.append({
        'Date': date,
        'Symbol': symbol,
        'BuySignal': buy_signal,
        'ExitSignal': exit_signal
    }, ignore_index=True)

# Results
output_filename = "output.csv"
output_df.to_csv(output_filename, index=False)

print("Results saved to", output_filename)

In [18]:
from kiteconnect import KiteConnect
import pandas as pd
import datetime as dt
import numpy as np

#API Creds
api_key = "jlqcagntp2b2pz1d"  
api_secret = "ph8nb7ale6oiym2upkuxlx7incp7p5mc"
access_token = "Fe78sU7fo3aF1PGNFYrP8YtHe2dTY3pf"
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

# Define the symbol and date range
symbol = 'AAPL'  # Replace with your desired symbol
start_date = dt.date(2023, 1, 1)
end_date = dt.date(2023, 8, 1)

# Fetch historical data using Kite API
data = kite.historical_data(
    symbol,
    start_date,
    end_date,
    interval="day"
)

# Convert data to DataFrame
df = pd.DataFrame(data)

# Calculate 20-period Exponential Moving Average (EMA)
period = 20
df['EMA'] = df['close'].ewm(span=period, adjust=False).mean()

# Generate buy signals
df['buy_signal'] = np.where(df['close'] > df['EMA'], 1, 0)

# Iterate through the DataFrame to find buy signals for the next week
for i in range(len(df) - 7):
    if df.loc[i, 'buy_signal'] == 1:
        next_week_candles = df.loc[i + 1:i + 7]
        if any(next_week_candles['low'] <= next_week_candles.loc[i, 'EMA']):
            print(f"Buy signal on {df.loc[i, 'date']} for {symbol}")

# Note: This is a simplified example and may require further refinement based on your specific use case.


InputException: invalid token